In [1]:
from crr_labels import fantom, roadmap
import os
from tqdm.auto import tqdm

In [2]:
cell_lines = [
    "GM12878", "HelaS3", "HepG2", "K562", "A549"
]

In [5]:
for window in tqdm((100, 200, 300, 500, 1000, 2000, 3000, 5000, 10000)):
    for center_enhancers in tqdm(("peak", "center")):
        path = f"preprocessed/fantom/window_size/{window}"
        if os.path.exists(path):
            continue
        enhancers, promoters = fantom(cell_lines, window, center_enhancers=center_enhancers)
        os.makedirs(path, exist_ok=True)
        enhancers.to_csv(f"{path}/enhancers_{center_enhancers}.csv")
        enhancers.to_csv(f"{path}/promoters.csv")
    for model in tqdm((15, 18)):
        path = f"preprocessed/roadmap/model/{model}/window_size/{window}"
        if os.path.exists(path):
            continue
        enhancers, promoters = roadmap(cell_lines, window)
        os.makedirs(path, exist_ok=True)
        enhancers.to_csv(f"{path}/enhancers.csv")
        enhancers.to_csv(f"{path}/promoters.csv")